# Replication of SpeechBrain-MOABB experimental results

## **Prerequisites**


### Install moabb

In [ ]:
%%capture
!pip install moabb

### Install braindecode and skorch

In [ ]:
%%capture
!pip install braindecode
!pip install skorch

## **Defining MOABB pipelines, evaluation scheme and dataset to use**
Here, we set up MOABB functionalities for running a benchmark on BNCI2014-001 dataset (also known as 'BCI IV2a' dataset). 
See a similar MOABB tutorial available at http://moabb.neurotechx.com/docs/auto_tutorials/tutorial_3_benchmarking_multiple_pipelines.html#sphx-glr-auto-tutorials-tutorial-3-benchmarking-multiple-pipelines-py.

The pipelines adopted are the ones provided by MOABB at: https://github.com/NeuroTechX/moabb/tree/develop/pipelines. 
Here, a cross-session evaluation scheme was adopted (i.e., leave-one-session-out cross-validation).



In [ ]:
import warnings
import moabb
import mne
import os
import numpy as np
from scipy.stats import sem
from moabb.pipelines.utils import create_pipeline_from_config
import yaml
from moabb.datasets import BNCI2014_001
from moabb.evaluations import CrossSessionEvaluation
from moabb.paradigms import MotorImagery
import pickle

mne.set_log_level("CRITICAL")
moabb.set_log_level("info")
warnings.filterwarnings("ignore")

In [ ]:
from collections import Counter
from functools import partial
from inspect import getmembers, isclass, isroutine

import mne
from braindecode.datasets.base import BaseConcatDataset
from braindecode.datasets.xy import create_from_X_y
from numpy import unique
from sklearn.base import BaseEstimator, TransformerMixin
from skorch.callbacks import Callback
from torch.nn import Module

class BraindecodeDatasetLoaderArray(BaseEstimator, TransformerMixin):
    def __init__(self, sfreq, ch_names=None, drop_last_window=False, kw_args=None):
        self.sfreq = sfreq
        self.ch_names = ch_names
        self.drop_last_window = drop_last_window
        self.kw_args = kw_args

    def fit(self, X, y=None):
        self.y = y
        return self

    def transform(self, X, y=None):
        if y is None:
            y = self.y
        dataset = create_from_X_y(
            X=X,
            y=y,
            window_size_samples=X.shape[2],
            window_stride_samples=X.shape[2],
            drop_last_window=self.drop_last_window,
            ch_names=self.ch_names,
            sfreq=self.sfreq,
        )

        return dataset

    def __sklearn_is_fitted__(self):
        """Return True since Transformer is stateless."""
        return True

In [ ]:
import matplotlib.pyplot as plt
import mne
import seaborn as sns
import torch
from braindecode import EEGClassifier
from braindecode.models import EEGNetv4, EEGInception, ShallowFBCSPNet, Deep4Net
from sklearn.pipeline import Pipeline
from skorch.callbacks import EarlyStopping, EpochScoring
from skorch.dataset import ValidSplit
from moabb.datasets import BNCI2014_001
from moabb.evaluations import CrossSessionEvaluation
from moabb.paradigms import MotorImagery
from moabb.pipelines.features import Resampler_Epoch
from moabb.utils import setup_seed

mne.set_log_level(False)

# Print Information PyTorch
print(f"Torch Version: {torch.__version__}")

# Set up GPU if it is there
cuda = torch.cuda.is_available()
device = "cuda" if cuda else "cpu"
print("GPU is", "AVAILABLE" if cuda else "NOT AVAILABLE")

# Set random seed to be able to reproduce results
seed = 42
setup_seed(seed)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Hyperparameter
LEARNING_RATE = 0.0001
WEIGHT_DECAY = 0
BATCH_SIZE = 64
SEED = 42
VERBOSE = 1
EPOCH = 1000
PATIENCE = 300

model = EEGNetv4(n_chans=22,n_outputs=4,n_times=4*125)
if device=='cuda':
    model = model.cuda()
clf = EEGClassifier(
    module=model,
    criterion=torch.nn.CrossEntropyLoss,
    optimizer=torch.optim.Adam,
    optimizer__lr=LEARNING_RATE,
    batch_size=BATCH_SIZE,
    max_epochs=EPOCH,
    train_split=ValidSplit(0.2, random_state=seed),
    device=device,
    callbacks=[
        EarlyStopping(monitor="valid_loss", patience=PATIENCE),
        EpochScoring(
            scoring="accuracy", on_train=True, name="train_acc", lower_is_better=False
        ),
        EpochScoring(
            scoring="accuracy", on_train=False, name="valid_acc", lower_is_better=False
        ),

    ],
    verbose=0,  # Not printing the results for each epoch
)

pipes = {}
pipes["EEGNetV4"] = Pipeline([
    ('convert', moabb.pipelines.features.Convert_Epoch_Array()),
    ('scaler', moabb.pipelines.features.StandardScaler_Epoch()),
    ("Braindecode_dataset", BraindecodeDatasetLoaderArray(sfreq=125)),
     ("EEGNetv4", clf)])

dataset = BNCI2014_001()
paradigm = MotorImagery(n_classes=4, tmin=0, tmax=4, fmin=4, fmax=40, resample=125) # no resampling, using @250 Hz
evaluation = CrossSessionEvaluation(
    paradigm=paradigm,
    datasets=dataset,
    suffix="braindecode_example",
    hdf5_path='/content',
    overwrite=True,
    return_epochs=True,
)

results_eegnet = evaluation.process(pipes)
results_eegnet.to_csv('/content/benchmark_bnci2014001_network.csv', sep=',')

In [ ]:
# ML pipelines with tuned hyper-parameters

tssvm_grid= """
name: Tangent Space SVM Grid

paradigms:
  - LeftRightImagery
  - MotorImagery

citations:
  - https://doi.org/10.1016/j.neucom.2012.12.039

pipeline:
  - name: Covariances
    from: pyriemann.estimation
    parameters:
      estimator: oas

  - name: TangentSpace
    from: pyriemann.tangentspace
    parameters:
      metric: "riemann"

  - name: SVC
    from: sklearn.svm
    parameters:
      kernel: "linear"

param_grid:
  svc__C:
    - 0.5
    - 1
    - 1.5
  svc__kernel:
    - "rbf"
    - "linear"
"""
f = open('/content/tssvm_grid.yaml', "w")
f.write(tssvm_grid)
f.close()

# ML pipelines with fixed hyper-parameters
csp_lda = """
name: CSP + LDA

paradigms:
  - LeftRightImagery
  - MotorImagery

citations:
  - https://doi.org/10.1007/BF01129656
  - https://doi.org/10.1109/MSP.2008.4408441

pipeline:
  - name: Covariances
    from: pyriemann.estimation
    parameters:
      estimator: oas

  - name: CSP
    from: pyriemann.spatialfilters
    parameters:
      nfilter: 6

  - name: LinearDiscriminantAnalysis
    from: sklearn.discriminant_analysis
    parameters:
      solver: svd
"""

f = open('/content/csp_lda.yaml', "w")
f.write(csp_lda)
f.close()

fcmdm= """
name: FgMDM

paradigms:
  - LeftRightImagery
  - MotorImagery

citations:
  - https://doi.org/10.1007/978-3-642-15995-4_78

pipeline:
  - name: Covariances
    from: pyriemann.estimation
    parameters:
      estimator: oas

  - name: FgMDM
    from: pyriemann.classification
    parameters:
      metric: "riemann"
"""
f = open('/content/fcmdm.yaml', "w")
f.write(fcmdm)
f.close()

mdm="""
name: MDM

paradigms:
  - LeftRightImagery
  - MotorImagery

citations:
  - https://doi.org/10.1109/TBME.2011.2172210

pipeline:
  - name: Covariances
    from: pyriemann.estimation
    parameters:
      estimator: oas

  - name: MDM
    from: pyriemann.classification
    parameters:
      metric: "riemann"
"""
f = open('/content/mdm.yaml', "w")
f.write(mdm)
f.close()

tslr="""
name: Tangent Space LR

paradigms:
  - LeftRightImagery
  - MotorImagery

citations:
  - https://doi.org/10.1016/j.neucom.2012.12.039

pipeline:
  - name: Covariances
    from: pyriemann.estimation
    parameters:
      estimator: oas

  - name: TangentSpace
    from: pyriemann.tangentspace
    parameters:
      metric: "riemann"

  - name: LogisticRegression
    from: sklearn.linear_model
    parameters:
      C: 1.0
"""
f = open('/content/tslr.yaml', "w")
f.write(tslr)
f.close()

regcsp_shlda="""
name: DLCSPauto + shLDA

paradigms:
  - LeftRightImagery
  - MotorImagery

citations:
  - https://doi.org/10.1007/BF01129656
  - https://doi.org/10.1109/MSP.2008.4408441

pipeline:
  - name: Covariances
    from: pyriemann.estimation
    parameters:
      estimator: oas

  - name: CSP
    from: pyriemann.spatialfilters
    parameters:
      nfilter: 6

  - name: LinearDiscriminantAnalysis
    from: sklearn.discriminant_analysis
    parameters:
      solver: lsqr
      shrinkage: auto
"""
f = open('/content/regcsp_shlda.yaml', "w")
f.write(regcsp_shlda)
f.close()

In [ ]:
pipeline_fpaths = ['/content/tssvm_grid.yaml',
                   '/content/csp_lda.yaml',
                   '/content/fcmdm.yaml',
                   '/content/mdm.yaml',
                   '/content/tslr.yaml',
                   '/content/regcsp_shlda.yaml',]
pipeline_names = ['TS+SVM',
                  'CSP+LDA',
                  'FgMDM',
                  'MDM',
                  'TS+LR',
                  'regCSP+shLDA']
pipelines = {}
print('Building pipelines...')
for pipeline_name, pipeline_fpath in zip(pipeline_names, pipeline_fpaths):
    #ppl = os.path.split(pipeline_fpath)[1].split('.yaml')[0]
    with open(pipeline_fpath) as f:
        pipeline = yaml.safe_load(f)
    print('Including {0} pipeline'.format(pipeline_name))
    pipelines[pipeline_name] = create_pipeline_from_config(pipeline['pipeline'])


In [ ]:
datasets = [BNCI2014_001()]
paradigm = MotorImagery(n_classes=4, resample=125, tmin=0, tmax=4, fmin=4, fmax=40, )
evaluation = CrossSessionEvaluation(
    paradigm=paradigm, datasets=datasets,
    hdf5_path='/content',
    overwrite=True
)

results = evaluation.process(pipelines)
results.to_csv('/content/benchmark_bnci2014001_ml.csv', sep=',')

In [ ]:
import pandas as pd
results_ml = pd.read_csv('/content/benchmark_bnci2014001_ml.csv', sep=',')
results_network = pd.read_csv('/content/benchmark_bnci2014001_network.csv', sep=',')

results = pd.concat([results_ml, results_network]).reset_index(drop=True)
results.to_csv('/content/benchmark_bnci2014001.csv', sep=',')

## **Extract the test set results and save results as dict**

In [ ]:
pipeline_names = list(pipelines.keys())
pipeline_names = sorted(pipeline_names)
pipeline_names += ['EEGNetV4']
print('Results on: \n-dataset: {0};\n-cross-session evaluation scheme;\n-score function: {1}.'.format(datasets[0].code, paradigm.scoring))
print('#'*10)
results_moabb = {}
for pipeline_name in pipeline_names:
    idx_pipeline = np.where(results['pipeline'].values==pipeline_name)[0]
    scores = results['score'].values[idx_pipeline]
    print('{0}: {1} (mean value); {2} (standard error of the mean)'.format(pipeline_name,
                                                                           round(np.mean(scores).item(),4),
                                                                           round(sem(scores).item(), 4)))
    results_moabb[pipeline_name] = results['score'][idx_pipeline].values.reshape(
        np.unique(results['subject']).shape[0],
        np.unique(results['session']).shape[0])
with open('/content/results_moabb.pkl', 'wb') as handle:
    pickle.dump(results_moabb, handle, protocol=pickle.HIGHEST_PROTOCOL)
print('#'*10)

## **Download and unzip SpeechBrain-MOABB results**

Now download the results obtained with SpeechBrain-MOABB using neural networks, namely EEGNet, ShallowConvNet, and EEGConformer. These networks were trained on many MOABB datasets (including also on BNCI2014-001) also searching for the optimal hyper-parameters of the entire decoding pipeline (data pre-processing, data augmentation, network architecture, and network training). See the related yaml files for the hyper-parameters and the hyper-parameter search spaces of these networks.

You can easily replicate the obtained results by performing training and evaluation again using the yaml file contained in the tar.gz compressed archive (best_hparams.yaml file).

Please note that the file is quite large (19 GB) and the download will take a while (<15 minutes).

In [ ]:
# Downloading Speechbrain-MOABB results using the Dropbox file provided in the Speechbrain-MOABB repository
url="https://www.dropbox.com/sh/ux0i0suljojonmb/AABsTBpEKCTmVE784yQw-WGMa?dl=1"
!wget -O /content/results_speechbrain_moabb.zip $url

In [ ]:
# unzipping Speechbrain-MOABB results (specifically for BNCI2014-001)
!unzip -j /content/results_speechbrain_moabb.zip results_BNCI2014001.tar.gz -d /content
!mkdir /content/results_speechbrain_moabb
!tar -xvzf results_BNCI2014001.tar.gz -C /content/results_speechbrain_moabb

## **Extract the test set results and save results as dict**

In [ ]:
import glob
import pickle

model_fnames = ['EEGNet', 'ShallowConvNet', 'EEGConformer']
results_speechbrain_moabb_folder = '/content/results_speechbrain_moabb'
def get_results_single_run(run_folder):
    seed_dir = glob.glob(os.path.join(run_folder, '*/'))[0]
    sbj_dirs = glob.glob(os.path.join(seed_dir, 'leave-one-session-out', '*/'))
    sbj_dirs = sorted(sbj_dirs)
    m = []
    for sbj_dir in sbj_dirs:
        sess_dirs = glob.glob(os.path.join(sbj_dir, '*/'))
        mm = []
        for sess_dir in sess_dirs:
            with open(os.path.join(sess_dir, 'test_metrics.pkl'), 'rb') as f:
                data = pickle.load(f)
            mm.append(np.array(data['acc']))
        m.append(mm)
    m = np.array(m)
    return m
results = {}
for model_fname in model_fnames:
    final_evaluation_folder = os.path.join(results_speechbrain_moabb_folder,
                                           'BNCI2014001',
                                           model_fname,
                                           'hopt',
                                           'best',)
    final_evaluation_folder = glob.glob(os.path.join(final_evaluation_folder, '*/'))
    final_evaluation_folder = sorted(final_evaluation_folder)
    final_evaluation_folder = final_evaluation_folder[0]
    print(final_evaluation_folder)

    tmp_results = []
    for run in np.arange(10):
        run_folder = os.path.join(final_evaluation_folder, 'run{0}'.format(run+1))
        tmp_results.append(get_results_single_run(run_folder))
    tmp_results = np.array(tmp_results)
    print(tmp_results.shape)
    results[model_fname] = tmp_results
with open('/content/results_speechbrain_moabb.pkl', 'wb') as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)

## **Replicate the statistical analysis (SpeechBrain-MOABB vs. MOABB vs. MOABB+braindecode pipelines) and the figure of the paper (Fig. 4)**

In [ ]:
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import wilcoxon, sem
from statsmodels.stats.multitest import multipletests
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 14#16

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize

with open('/content/results_moabb.pkl', 'rb') as f:
    results_moabb = pickle.load(f)
with open('/content/results_speechbrain_moabb.pkl', 'rb') as f:
    results_speechbrain_moabb = pickle.load(f)
scores = []

target_keys_moabb = ['CSP+LDA', 'regCSP+shLDA', 'MDM', 'FgMDM', 'TS+LR', 'TS+SVM', 'EEGNetV4']
for key in target_keys_moabb:
    scores.append(np.mean(results_moabb[key], axis=-1))
for key in results_speechbrain_moabb.keys():
    scores.append(np.mean(results_speechbrain_moabb[key], axis=(0,-1)))

means = [np.mean(s) for s in scores]
sems = [sem(s) for s in scores]
keys = target_keys_moabb + list(results_speechbrain_moabb.keys())

fig, axs = plt.subplots(2,1, figsize=(8,11/1.25))
ax = axs[0]
vp = ax.violinplot(dataset=scores, widths=0.3,
                   showmeans=False, showmedians=False, showextrema=False,
                   )
for pc in vp['bodies']:
    pc.set_facecolor('#D43F3A')
    pc.set_edgecolor('black')
    pc.set_alpha(1)
    pc.set_zorder(3)
for pc in vp['bodies'][:len(target_keys_moabb)]:
    pc.set_facecolor('darkcyan')
    pc.set_edgecolor('black')
    pc.set_alpha(1)
    pc.set_zorder(3)
for pc in vp['bodies'][len(target_keys_moabb)-1:len(target_keys_moabb)]:
    pc.set_facecolor('darkorange')
    pc.set_edgecolor('black')
    pc.set_alpha(1)
    pc.set_zorder(3)
ax.errorbar(np.arange(len(keys))+1, means, sems, linestyle='none', marker='o',
            c='k', zorder=3)

ax.set_ylim(0., 1)
ax.axhline(y=0.25, c='k')
ax.grid(axis='y', c='k', linestyle='--', zorder=1)
ax.set_title('Performance validation on BNCI2014-001:\nMOABB vs. MOABB+braindecode vs. SpeechBrain-MOABB pipelines')
ax.set_xticks(np.arange(len(keys))+1)
ax.set_xticklabels(keys, rotation=30, ha='right')
ax.set_ylabel('Accuracy')
legend_elements = [
    Patch(facecolor='darkcyan', edgecolor='k', alpha=1, label='MOABB', hatch=""),
    Patch(facecolor='darkorange', edgecolor='k', alpha=1, label='MOABB+braindecode', hatch=""),
    Patch(facecolor='#D43F3A', edgecolor='k', alpha=1, label='SpeechBrain-MOABB', hatch=""),
]
leg = ax.legend(handles=legend_elements,
                  handlelength=2,
                  handletextpad=0.2,
                  labelspacing=0,
                  borderpad=0.3,
                  borderaxespad=0.1,
                  columnspacing=0.5,
                  ncols=1,
                  loc="lower right",
                  framealpha=1,
                  edgecolor="k",
                  ncol=1,
                  )

leg.get_frame().set_boxstyle('Round', pad=0., rounding_size=0)


ax = axs[1]
model_fname = 'EEGNet'
idx0 = [i for i,k in enumerate(keys) if k==model_fname][0]
diff_scores = [scores[idx0]-s for s in scores]
means = [np.mean(s) for s in diff_scores]
sems = [sem(s) for s in diff_scores]
vp = ax.violinplot(dataset=diff_scores, widths=0.3,
                   showmeans=False, showmedians=False, showextrema=False,
                   )

for pc in vp['bodies']:
    pc.set_facecolor('#D43F3A')
    pc.set_edgecolor('black')
    pc.set_alpha(1)
    pc.set_zorder(3)
for pc in vp['bodies'][:len(target_keys_moabb)]:
    pc.set_facecolor('darkcyan')
    pc.set_edgecolor('black')
    pc.set_alpha(1)
    pc.set_zorder(3)
for pc in vp['bodies'][len(target_keys_moabb)-1:len(target_keys_moabb)]:
    pc.set_facecolor('darkorange')
    pc.set_edgecolor('black')
    pc.set_alpha(1)
    pc.set_zorder(3)
idx = np.arange(len(keys))
idx = np.setdiff1d(idx, [idx0])

x = np.arange(len(keys))+1
ax.errorbar(x, np.array(means), np.array(sems), linestyle='none', marker='o',
            c='k', zorder=3)
ax.set_ylim(-0.1, 0.5)
ax.axhline(y=0., c='k')
ax.grid(axis='y', c='k', linestyle='--', zorder=1)
ax.set_xticks(np.arange(len(keys))+1)
ax.set_xticklabels(keys, rotation=30,ha='right')
ax.set_ylabel(r'$\Delta_{acc}$ (top decoder - others)')

d0 = np.mean(results_speechbrain_moabb[model_fname], axis=(0, -1)) # (9,)
pvals = []
conds = []
for pipeline_fname in target_keys_moabb:
    d1 = np.mean(results_moabb[pipeline_fname], axis=-1) # (9,)
    idx1 = [i for i,k in enumerate(keys) if k==pipeline_fname][0]
    print('{0} vs. {1}: difference={2}'.format(model_fname, pipeline_fname, np.mean(d0-d1)))
    _, p = wilcoxon(d0, d1)
    pvals.append(p)
    conds.append(idx1)
for pipeline_fname in results_speechbrain_moabb.keys():
    d1 = np.mean(results_speechbrain_moabb[pipeline_fname], axis=(0,-1)) # (9,)
    idx1 = [i for i,k in enumerate(keys) if k==pipeline_fname][0]
    if d0.mean()!=d1.mean():
        print('{0} vs. {1}: difference={2}'.format(model_fname, pipeline_fname, np.mean(d0-d1)))
        _, p = wilcoxon(d0, d1)
        pvals.append(p)
        conds.append(idx1)

print(conds)
print("Uncorrected p-values: {0}".format(pvals))
_, pvals, _, _ = multipletests(pvals, method='fdr_bh')
print("Corrected p-values for multiple tests: {0}".format(pvals))
signs = []
for p in pvals:
    if p<0.001:
        signs.append('***')
    else:
        if p<0.01:
            signs.append('**')
        else:
            if p<0.05:
                signs.append('*')
            else:
                signs.append('')
for k, sign in zip(conds, signs):
    ax.text(x=k+1, y=.4, s=sign, ha='center', va='bottom', fontsize=12)

fig.tight_layout()
fig.savefig('fig4.pdf', format='PDF', dpi=600)

In [ ]:
# Check for statistical significance vs. MOABB and MOABB+braindecode for each random initialization of EEGNet SpeechBrain-MOABB (not only for the average across seeds)
for i in np.arange(10):
    d0 = np.mean(results_speechbrain_moabb[model_fname][i,...], axis=(-1, )) # (9,)
    pvals = []
    for pipeline_fname in target_keys_moabb:
        d1 = np.mean(results_moabb[pipeline_fname], axis=-1) # (9,)
        idx1 = [i for i,k in enumerate(keys) if k==pipeline_fname][0]
        _, p = wilcoxon(d0, d1)
        pvals.append(p)
        conds.append(idx1)
    for pipeline_fname in results_speechbrain_moabb.keys():
        d1 = np.mean(results_speechbrain_moabb[pipeline_fname][i,...], axis=(-1, )) # (9,)
        idx1 = [i for i,k in enumerate(keys) if k==pipeline_fname][0]
        if d0.mean()!=d1.mean():
            _, p = wilcoxon(d0, d1)
            pvals.append(p)
            conds.append(idx1)
    _, pvals, _, _ = multipletests(pvals, method='fdr_bh')
    #print("Corrected p-values for multiple tests: {0}".format(pvals))
    print("random iteration no. {0} always significant vs. MOABB / MOABB+braindecode?: {1} ({2} comparisons corrected for multiple tests)".format(
        i, np.all(pvals[:-2]<.05), pvals.shape[0])
    )